Notice that I am assuming you have the relevant API_KEYs as environmental variables.

In [2]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
import os
import subprocess
from IPython.display import Markdown, display
from llama_index.core import PromptTemplate

## Defining global variables

In [3]:
COMPANIES = ["uber"]
DATA_DIRS = {}
PERSIST_DIRS = {}
for c in COMPANIES:
    DATA_DIRS[c] = os.path.join(os.environ["DATA_DIR"], f"{c}")
    PERSIST_DIRS[c] = os.path.join(os.environ["PERSIST_DIR"], f"{c}")

## Download data
You can access more files by usinig the next url format

https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/{`COMPANY_NAME`}_10q_{`MONTH`}_{`YEAR`}.pdf'

In [4]:
for c in COMPANIES:
    if not os.path.exists(DATA_DIRS[c]):
        os.mkdir(DATA_DIRS[c])
        command = f"wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/{c}_2021.pdf' -O '{DATA_DIRS[c]}/{c}_2021.pdf'"        
        subprocess.run(command, shell=True)

# Create index and query engine for each company individually

In [5]:
query_engine_dict = {}
for c in COMPANIES:
    if not os.path.exists(PERSIST_DIRS[c]):
        print("Creating Index")
        # load the documents and create the index
        documents = SimpleDirectoryReader(DATA_DIRS[c]).load_data()
        index = VectorStoreIndex.from_documents(documents)
        # store it for later
        index.storage_context.persist(persist_dir=PERSIST_DIRS[c])
    else:
        print("Loading Index")
        # load the existing index
        storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIRS[c])
        index = load_index_from_storage(storage_context)
    
    query_engine_dict[c] = index.as_query_engine()

Loading Index


## Define prompt viewing function

In [7]:
def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}<br>" f"**Text:** <br>"
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown("<br><br>"))


## Accessing Prompts

Here we get the prompts from the query engine. Note that all prompts are returned, including ones used in sub-modules in the query engine. This allows you to centralize a view of these prompts!


In [6]:
prompts_dict = query_engine_dict["uber"].get_prompts()
display_prompt_dict(prompts_dict)

**Prompt Key**: response_synthesizer:text_qa_template<br>**Text:** <br>

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


<br><br>

**Prompt Key**: response_synthesizer:refine_template<br>**Text:** <br>

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


<br><br>

In [8]:
response = query_engine_dict["uber"].query("How has total revenue changed in 2019, 2020 and 2021?")
print(response)

Total revenue changed from $11,139 million in 2019 to $17,455 million in 2021, representing a 57% increase from 2020 to 2021.



## Customize the prompt

You can also update/customize the prompts with the update_prompts function. Pass in arg values with the keys equal to the keys you see in the prompt dictionary.

Here we'll change the text_qa_template to generate response in both english and Spanish.


In [11]:
new_tmpl_str = (
        "Below is the specific context required to answer the upcoming query. You must base your response solely on this context, strictly avoiding the use of external knowledge or assumptions..\n"
        "---------------------\n"
        "{context_str}\n"
        "---------------------\n"
        "Given this context, please formulate your response to the following query. It is imperative that you respond in Spanish and English.\n"
        "Query: {query_str}\n"
        "Answer: "
    )
new_tmpl = PromptTemplate(new_tmpl_str)
query_engine_dict["uber"].update_prompts({"response_synthesizer:text_qa_template": new_tmpl})

In [12]:
prompts_dict = query_engine_dict["uber"].get_prompts()
display_prompt_dict(prompts_dict)

**Prompt Key**: response_synthesizer:text_qa_template<br>**Text:** <br>

Below is the specific context required to answer the upcoming query. You must base your response solely on this context, strictly avoiding the use of external knowledge or assumptions..
---------------------
{context_str}
---------------------
Given this context, please formulate your response to the following query. It is imperative that you respond in Spanish and English.
Query: {query_str}
Answer: 


<br><br>

**Prompt Key**: response_synthesizer:refine_template<br>**Text:** <br>

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


<br><br>

In [13]:
response = query_engine_dict["uber"].query("How has total revenue changed in 2019, 2020 and 2021?")
print(response)

Total revenue has changed as follows:
- In 2019, the revenue was not provided in the context.
- In 2020, the revenue was $11,139 million.
- In 2021, the revenue increased to $17,455 million, representing a 57% increase compared to 2020. 

El total de ingresos ha cambiado de la siguiente manera:
- En 2019, los ingresos no se proporcionaron en el contexto.
- En 2020, los ingresos fueron de $11,139 millones.
- En 2021, los ingresos aumentaron a $17,455 millones, lo que representa un aumento del 57% en comparación con 2020.
